In [ ]:
import json
import requests
import pandas as pd

## Get course realization data from course codes or keywords

In [ ]:
# Headers to use in requests
with open('headers.json', 'r') as f:
    headers = json.load(f)

In [ ]:
course_category = 'method' # method / datafication / general
query_type = 'codes' # codes / keywords

In [ ]:
# Read query terms from file
with open('query-terms/' + query_type + '_' + course_category + '.txt', 'r') as f:
    query_terms = [l.strip() for l in f.readlines()]

query_terms = list( dict.fromkeys(query_terms) )  # remove duplicates
print( len(query_terms) )

In [ ]:
# Function for downloading realizations for each query term, given query type
def get_realizations(query_term, query_type, headers):
    
    if query_type == 'codes':
        search_string = 'codeQuery:"' + query_term + '"'
    else:
        search_string = 'fullTextQuery:"' + query_term + '"'
    
    # Filter external courses
    search_string = search_string + ', universityOrgIds:["hy-university-root-id"]'
    
    # Construct query for Sisu API
    query = '{ course_unit_search( ' + search_string + ' ) '
    query += '{ id code name{fi} courseUnitRealisations{ '
    query += 'id name{fi} documentState nameSpecifier{fi} courseUnitRealisationType{'
    query += 'name{fi}} activityPeriod{ startDate endDate } '
    query += '} '
    query += '} }'
            
    json_data = { 'query': query }
    response = requests.post( headers['Origin'] + '/api/', headers=headers, json=json_data )
    
    course_data = response.json()['data']['course_unit_search']
    
    if not course_data:
        print(search_string, "returned no data.")
    else:
        return( course_data )

In [ ]:
# Get all course data for query terms
course_data = [get_realizations(term, query_type, headers) for term in query_terms]

# Clean list
course_data = [c for c in course_data if c is not None]

# Flatten list
course_data = [c for course in course_data for c in course]

len( course_data )

In [ ]:
# Get course realizations from course data
realizations = []
for course in course_data:
    
    for realization in course['courseUnitRealisations']:
        
        # Discard inactive realizations
        if (realization['documentState'] == 'ACTIVE'):
            
            code = course['code']
            study_unit_name = course['name']['fi']
            realization_id = realization['id']
            realization_name = realization['name']['fi']
            realization_name_specifier = realization['nameSpecifier']['fi']
            realization_type = realization['courseUnitRealisationType']['name']['fi']
            realization_start = realization['activityPeriod']['startDate']
            realization_end = realization['activityPeriod']['endDate']
            realization_url = 'https://studies.helsinki.fi/opintotarjonta/cur/' + realization['id']
                        
            realizations.append(
                (
                    realization_id,
                    code,
                    study_unit_name,
                    realization_name,
                    realization_name_specifier,
                    realization_type,
                    realization_url,
                    realization_start,
                    realization_end
                )
            )

In [ ]:
columns = ['id', 'code', 'study_unit_name', 'name', 'name_specifier', 'type', 'url', 'start', 'end']
df_realizations = pd.DataFrame(realizations, columns=columns)

In [ ]:
df_realizations.to_json('data/' + course_category + '_' + query_type + '_realizations.json')

# Filter course realizations

In [ ]:
df_realizations = pd.read_json('data/' + course_category + '_' + query_type + '_realizations' + '.json')

In [ ]:
# Filter course realizations by start date
df_realizations['start'] = df_realizations['start'].apply(pd.to_datetime)
df_realizations['end'] = df_realizations['end'].apply(pd.to_datetime)
df_realizations = df_realizations[df_realizations['start'] > pd.to_datetime('2023-07-01')]

In [ ]:
# Drop duplicates, nan results and redundant columns
df_realizations.drop_duplicates(subset=['id','name','type','url','start','end'], keep='last', inplace=True)
df_realizations.dropna(subset=['id','name'])
df_realizations = df_realizations.drop('id', axis=1)

In [ ]:
# Check course types in data
df_realizations['type'].value_counts()

In [ ]:
# Select relevant course types
course_types = ['Luento-opetus', 'Verkko-opetus', 'Laboratoriotyöskentely', 'Lähiopetus', 'Projekti',
                'Etäopetus','Seminaari','Kirjallinen työ','Pienryhmäopetus','Itsenäinen projekti',
                'Monimuoto-opetus']

mask_type = df_realizations['type'].apply( lambda t: t in course_types )

df_realizations = df_realizations[mask_type]

In [ ]:
# Get courses with unclear names
unclear_names = ['Verkko-opetus','Luento-opetus','Online teaching','Itsenäinen työskentely',
                 'Kirjallinen työ', 'Avoin yliopisto','Laboratoriotyöskentely',
                 "Lähiopetus","Projekti","Toteutus","Syksy 2023","Harjoitukset ja videoluennot",
                 'Verkko-opetus, etäopetus Kokkolassa', 'Verkko-opetus, avoin yliopisto-opetus (Seinäjoki)',
                 "Lectures and Laboratory work", "Luennot, harjoitukset & projekti",
                 "Metodologian praktikum (KUMA-EF301), Lähiopetus", "Luento-opetus, Chydeniuksen opiskelijoille",
                 "Kevään toteutus (FI/EN)", "Syksyn toteutus (FI/EN)", "Kesän verkko-opetus (FI/EN)",
                 "Luento-opetus, suomeksi","Luento-opetus, englanniksi","Verkko-opetus, LV 2023-2024",
                 "Ei opetusta lv 2023-2024 Data Mining: The B-Course and GUHA-Method"]

unclear_names = [n.lower() for n in unclear_names]

# If course has unclear name or name is identical to course code, mark as unclear
mask_unclear = df_realizations.apply( 
    lambda course: (course['name'].lower() in unclear_names) | (course['name'].lower() == course['code'].lower()),
    axis=1
)

df_unclear = df_realizations[mask_unclear]

In [ ]:
# Replace unclear names with name specifiers
df_unclear.loc[:,'name'] = df_unclear['name_specifier']
df_unclear = df_unclear.drop('name_specifier', axis=1)

# Check if the corrected name is the same as course type; if yes, replace with study unit name
course_name_type_mask = df_unclear.apply( lambda course: course['name'].lower() == course['type'].lower(), axis=1)
df_unclear.loc[course_name_type_mask,'name'] = df_unclear['study_unit_name']

# Write courses with unclear names to json
df_unclear['start'] = df_unclear['start'].astype('str')
df_unclear['end'] = df_unclear['end'].astype('str')
df_unclear.to_json('data/' + course_category + '_' + query_type + '_unclear.json')

In [ ]:
# Drop courses with unclear names
df_realizations = df_realizations[~mask_unclear]
df_realizations.reset_index(drop=True, inplace=True)

In [ ]:
# Write filtered course realizations to json, or directly to xlsx for general courses
df_realizations['start'] = df_realizations['start'].astype('str')
df_realizations['end'] = df_realizations['end'].astype('str')
df_realizations.to_json('data/' + course_category + '_' + query_type + '_filtered_realizations.json')

if course_category == 'general':
    df_realizations.to_excel(course_category + '_courses_23-24.xlsx', index=False)

# Combine and deduplicate data and write to xlsx

In [ ]:
course_category = 'method'

In [ ]:
# Get data for course codes and keywords
df_codes = pd.read_json('data/' + course_category + '_codes_filtered_realizations.json')
df_keywords = pd.read_json('data/' + course_category + '_keywords_filtered_realizations.json')

df_codes_unclear = pd.read_json('data/' + course_category + '_codes_unclear.json')
df_keywords_unclear = pd.read_json('data/' + course_category + '_keywords_unclear.json')

In [ ]:
# Combine and deduplicate data from codes and keywords, write to excel
df_combined = pd.concat([df_codes,df_keywords], ignore_index=True)
df_combined.drop_duplicates(inplace=True)
df_combined.drop('name_specifier', axis=1, inplace=True)
print( len(df_combined) )

df_combined.to_excel(course_category + '_courses_23-24.xlsx', index=False)

In [ ]:
# Combine and deduplicate unclear data, write to excel
df_combined_unclear = pd.concat([df_codes_unclear,df_keywords_unclear], ignore_index=True)
df_combined_unclear.drop_duplicates(inplace=True)
print( len(df_combined_unclear) )

df_combined_unclear.to_excel(course_category + '_unclear.xlsx', index=False)

### Check general data against other datasets and drop duplicates

In [ ]:
# Get data for method and datafication query terms
df_method = pd.read_excel('method_courses_23-24.xlsx')
df_datafication = pd.read_excel('datafication_courses_23-24.xlsx')
df_method_datafication = pd.concat([df_method,df_datafication], ignore_index=True)

# Get data for general query terms
df_general = pd.read_excel('general_courses_23-24.xlsx')
print( len(df_general) )

# Drop courses from general data that are in method or datafication courses
df_merge = pd.merge(df_general, df_method_datafication, how='left', indicator='exist')
df_merge = df_merge[df_merge['exist'] != 'both']

df_merge.drop('exist', axis=1, inplace=True)
df_merge.drop('name_specifier', axis=1, inplace=True)

print( len(df_merge) )

In [ ]:
df_merge.to_excel('general_courses_23-24.xlsx', index=False)

In [ ]:
# Get data with unclear names for method and datafication query terms
df_method_unclear = pd.read_excel('method_unclear.xlsx')
df_datafication_unclear = pd.read_excel('datafication_unclear.xlsx')
df_method_datafication_unclear = pd.concat([df_method_unclear,df_datafication_unclear], ignore_index=True)

# Get data with unclear names for general query terms
df_general_unclear = pd.read_json('data/general_keywords_unclear.json')
print( len(df_general_unclear) )

# Drop data that are in method or datafication courses
df_merge_unclear = pd.merge(df_general_unclear, df_method_datafication_unclear, how='left', indicator='exist')
df_merge_unclear = df_merge_unclear[df_merge_unclear['exist'] != 'both']

df_merge_unclear.drop('exist', axis=1, inplace=True)

print( len(df_merge_unclear) )

In [ ]:
df_merge_unclear.to_excel('general_unclear.xlsx', index=False)